In [ ]:
%matplotlib notebook
import numpy as np
import scipy.signal
import scipy.fft as sfft
import matplotlib.pylab as plt
from matplotlib import animation

from IPython.display import YouTubeVideo, HTML, Audio
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show, output_notebook
output_notebook()

# Estimadores adaptivos parte II

## Algoritmo Recursive Least Squares (RLS)



El algoritmo LMS
- minimiza el error instantaneo
- es simple y eficiente 
- en algunos casos su convergencia es demasiado lenta

Podemos obtener un filtro adaptivo que converge más rápido si reemplazamos el error instantaneo por el error histórico

Sigamos considerando un filtro tipo FIR con $L+1$ pesos que se actualizan de en cada época

$$
y_i = \sum_{k=0}^L w_{i, k} u_{n-k}
$$

El algoritmo RLS (recursive least squares) es un método online que minimiza el error histórico, es decir la suma de errores entre la muestra actual y la inicial

$$
\begin{align}
J^H_n(\textbf{w}) &= \sum_{i=L}^n   \beta^{n-i} |e_i|^2 \nonumber \\
&= \sum_{i=L+1}^n \beta^{n-i} (d_i - \sum_{k=0}^{L} w_{i, k} u_{i-k} )^2, \nonumber
\end{align}
$$

donde $n$ es el índice del instante actual y $\beta \in [0, 1]$ es el "factor de olvido" y que usualmente es un valor cercano a $1$ 

Adicionalmente se agrega un regularizador a los pesos
$$
J^w_n = \delta  \| \textbf{w}_{n} \|^2
$$

La solución cerrada sería

$$
\textbf{w}_n = (U_n^T \pmb{\beta} U_n + \delta I)^{-1}  U_n^T \pmb{\beta} \textbf{d}_n
$$
donde 
$$
\textbf{d}_n = \begin{pmatrix}  d_n \\ d_{n-1} \\ \vdots \\ d_{L+1} \end{pmatrix} \quad
\textbf{u}_n = \begin{pmatrix}  u_n \\ u_{n-1} \\ \vdots \\ u_{n-(L+1)} \end{pmatrix} \quad
\pmb{\beta} = I \begin{pmatrix} \beta \\ \beta^{1} \\ \beta^{2}  \vdots \\ \beta^{n-L-1} \end{pmatrix}
\quad 
U_n = \begin{pmatrix}
\textbf{u}_n^T \\ \textbf{u}_{n-1}^T \\ \vdots \\ \textbf{u}_{L+1}^T \\
\end{pmatrix} \in \mathbb{R}^{n - (L+1) \times L+1}
$$

e $I$ es la matriz identidad. 

- Notemos la similitud con el filtro de Wiener
    - Matriz de correlación ponderada y regularizada: $\Phi_n = U_n^T \pmb{\beta} U_n + \delta I$
    - Vector de correalación cruzada ponderada:  $\theta_n = U_n^T \pmb{\beta} \textbf{d}_n$
- Queremos recomputar esta solución cuando llegan nuevas observaciones
- En particular queremos evitar invertir $\Phi_n$

El algoritmo **RLS** propone una solución que actualiza los pesos de forma recursiva

Las condiciones iniciales son 
- $\Phi_0 = \delta I$
- $\theta_0 = 0$

y luego la actualización viene dada por 

- $\Phi_{n} = \beta \Phi_{n-1} + \textbf{u}_n \textbf{u}_n^T$ 
- $\theta_{n} = \beta \theta_{n-1} + \textbf{u}_n d_n $ 
- $\textbf{w}_n = \Phi_n^{-1} \theta_n$

Que es más eficiente si actualizamos $\Phi_{n}^{-1}$ en lugar de $\Phi_{n}$ 

Usando el lema de inversión de matrices 
$$
(A + UCV)^{-1} = A^{-1} - A^{-1} U (C^{-1} + VA^{-1} U)^{-1} V A^{-1}
$$

con $A = \Phi_{n-1}^{-1}$, $C=1$, $U= \textbf{u}_n$ y $V = \textbf{u}_n^T$ entonces

$$
\begin{align}
\Phi_{n}^{-1} &= \left(\beta \Phi_{n-1} + \textbf{u}_n \textbf{u}_n^T \right)^{-1} \nonumber \\
&= \beta^{-1} \Phi_{n-1}^{-1} - \beta^{-2} \frac{\Phi_{n-1}^{-1} \textbf{u}_n \textbf{u}_n^T \Phi_{n-1}^{-1} }{1 + \beta^{-1} \textbf{u}_n^T \Phi_{n-1}^{-1} \textbf{u}_n} \nonumber \\
&= \beta^{-1} \Phi_{n-1}^{-1} - \beta^{-1} \textbf{k}_n \textbf{u}_n^T \Phi_{n-1}^{-1}, \nonumber 
\end{align}
$$

donde llamamos **ganancia** a $\textbf{k}_n$

Podemos continuar para encontrar una regla de actualización recursiva para los pesos

$$
\begin{align}
\textbf{w}_n &= \Phi_n^{-1} \theta_n \nonumber \\
&=  \Phi_n^{-1} \beta \theta_{n-1} + \Phi_n^{-1} \textbf{u}_n d_n\nonumber \\
&=  \Phi_{n-1}^{-1} \theta_{n-1} - \textbf{k}_n \textbf{u}_n^T \Phi_{n-1}^{-1} \theta_{n-1} + \Phi_n^{-1} \textbf{u}_n d_n \nonumber \\
&=  \textbf{w}_{n-1} - \textbf{k}_n \textbf{u}_n^T  \textbf{w}_{n-1} + \Phi_n^{-1} \textbf{u}_n d_n \nonumber \\
&=  \textbf{w}_{n-1} + \textbf{k}_n ( d_n - \textbf{u}_n^T  \textbf{w}_{n-1} ) \nonumber \\
&=  \textbf{w}_{n-1} + \textbf{k}_n e_n \nonumber 
\end{align}
$$
donde reemplazamos $\textbf{w}_{n-1} = \Phi_{n-1}^{-1} \theta_{n-1}$ y usamos que 
$$
\begin{align}
\textbf{k}_n &= \left(\beta^{-1} \Phi_{n-1}^{-1} - \beta^{-1} \textbf{k}_n \textbf{u}_n^T \Phi_{n-1}^{-1} \right)  \textbf{u}_n \nonumber \\ &= \Phi_n^{-1} u_n \nonumber
\end{align}
$$


### Notas
- Con esto tenemos un algoritmo de complejidad $L^2$ en vez de $L^3$
- Esto sigue siendo mayor que LMS (complejidad $L$) pero con la ventaja de converger más rapidamente 
- En la literatura suele usarse el nombre $P_n$ para $\Phi_n^{-1}$



### Resumen algoritmo RLS

- Inicializar $\Phi_0 = \delta I$ y $\textbf{w}_0 = 0$
- Para $n \in [1, \infty]$
    1. Calcular la ganancia
    $$
    \textbf{k}_n =  \frac{\beta^{-1} \Phi_{n-1}^{-1} \textbf{u}_n }{1 + \beta^{-1} \textbf{u}_n^T \Phi_{n-1}^{-1} \textbf{u}_n}
    $$
    1. Calcular el error
    $$
    e_n = d_n - \textbf{u}_n^T  \textbf{w}_{n-1} 
    $$
    1. Actualizar el error de pesos
    $$
    \textbf{w}_n = \textbf{w}_{n-1} + \textbf{k}_n e_n 
    $$
    1. Actualizar el inverso de la matriz de correlación
    $$
    \Phi_{n}^{-1} = \beta^{-1} \Phi_{n-1}^{-1} - \beta^{-1} \textbf{k}_n \textbf{u}_n^T \Phi_{n-1}^{-1}
    $$
    
### Recomendaciones
- A menor $\delta$ mayor regularización. Usar $\delta$ pequeño para SNR bajo y $\delta$ grande para SNR alto
- Considerar un valor de $\beta \approx 0.9$ inicialmente
- Calibre $\delta$ y $\beta$ usando validación cruzada
***

### RLS versus LMS: Tracking

- Notemos la diferencia en tiempo de convergencia entre los algoritmos LMS y RLS
- RLS es capaz de adaptarse a cambios bruscos más rápido que LMS

In [ ]:
fig, ax = plt.subplots(2, figsize=(9, 6))
t, dt = np.linspace(0, 5, num=500, retstep=True)
np.random.seed(0)
u = np.sin(2.0*np.pi*t*5)  
# u[250:] += 5
u += np.array([5*np.exp(-np.absolute(tt-1)/0.2) if tt>1 else 0 for tt in t])
u += np.array([5*np.exp(-np.absolute(tt-3)/0.2) if tt>3 else 0 for tt in t])
u_noisy = u + 0.5*np.random.randn(len(t))

def update(mu, beta, L):
    ax[0].cla(); ax[1].cla(); 
    u_pred = np.zeros(shape=(len(u_noisy), 2))
    #LMS
    w = np.zeros(shape=(L+1, ))
    for k in range(L+1, len(u_noisy)):
        u_window = u_noisy[k-L-1:k]
        norm = np.sum(u_window**2) + 1e-6
        u_pred[k, 0] = np.dot(w, u_window)
        if k < len(u):
            w += 2*mu*(u_noisy[k] - u_pred[k, 0])*u_window/norm
    #RLS
    w = np.zeros(shape=(L+1, ))
    delta = 1.
    Phi_inv = delta*np.eye(L+1); 
    for k in range(L+1, len(u_noisy)):
        u_window = u_noisy[k-L-1:k]
        # Calcular ganancia
        gain = np.dot(Phi_inv, u_window)/(beta + np.dot(np.dot(u_window.T, Phi_inv), u_window))        
        # Calcular error
        u_pred[k, 1] = np.dot(w, u_window)
        err = u_noisy[k] - u_pred[k, 1]
        # Actualizar pesos
        w += np.dot(gain, err)
        # Actualizar el inverso de la matriz de correlación
        Phi_inv = (1./beta)*(1. - np.sum(gain*u_window))*Phi_inv
    
    ax[0].plot(t, u_noisy, 'k.', alpha=0.5); 
    ax[0].plot(t, u, 'g-', alpha=0.5, label='Real');  
    ax[0].plot(t, u_pred[:, 0], label='LMS'); 
    ax[0].plot(t, u_pred[:, 1], label='RLS'); ax[0].legend(loc=1)
    ax[1].plot((u - u_pred[:, 0])**2, label='LMS')
    ax[1].plot((u - u_pred[:, 1])**2, label='RLS'); ax[1].legend()

    
interact(update, 
         mu=FloatSlider_nice(step=0.01, max=0.1, min=0.01),
         beta=FloatSlider_nice(step=0.01, max=1., min=0.5, value=0.9),
         L=SelectionSlider_nice(options=[1, 5, 10, 20, 50], value=10));

## Algoritmo Perceptrón (Rosemblatt 1962)


- Podemos entrenar un filtro adaptivo para hacer **clasificación binaria de patrones**
- En este caso la respuesta deseada tiene dos categorías: $d_n \in \{-1, +1\}$ 
- La entrada se considera continua y de $M$ dimensiones: $u_n \in \mathbb{R}^M$
- Asumimos que se tienen $N$ tuplas $(u_n, d_n)$
- El filtro tiene arquitectura FIR con $M+1$ coeficientes pero se agrega una no linealidad $\phi(\cdot)$ en la salida
$$
\begin{align}
y_n &=  \phi \left(w_0 + \sum_{k=1}^{M} w_k u_{nk} \right) \nonumber \\
&= \phi \left(\langle \textbf{w}, \text{concat}(1, u_n) \rangle \right), \nonumber 
\end{align}
$$
donde podemos usar $\phi(z) = \text{sign}(z) = \begin{cases} +1 & z > 0 \\0 & z=0\\-1 & z<0 \end{cases}$
- Esto se conoce como el modelo matemático de una neurona de [McCulloch y Pitts](https://link.springer.com/article/10.1007/BF02478259)
    - Las coeficientes del filtro son los pesos sinápticos de las dendritas
    - La función no lineal corresponde al axón

<img src="../images/adaptive-neuron.png" width="400">

<img src="../images/adaptive-neuron2.jpeg" width="400">

- El algoritmo para entrenar la neurona artificial se conoce como **algoritmo percetrón**
- Asumimos un vector de pesos inicial nulo $\textbf{w}^{(t=0)} = [0, 0, ..., 0]$
- La función de costo en el instante $t$ al presentar el ejemplo $(d_n, u_n)$ 
$$
\mathcal{L}( \textbf{w}^{(t)} ) = \text{max} \Big(0 ~, - d_n \langle \textbf{w}^{(t)}, \text{concat}(1, u_n) \rangle \Big)
$$
- y su derivada es 
$$
\frac{d \mathcal{L}(\textbf{w}^{(t)} )}{d \textbf{w}}  = \begin{cases} 0 & d_n \langle \textbf{w}, \text{concat}(1, u_n) \rangle \geq 0 \\ - d_n \text{concat}(1, u_n)  & d_n \langle \textbf{w}, \text{concat}(1, u_n) \rangle < 0
\end{cases}
$$
es decir que la derivada es cero si el ejemplo está bien clasificado
- Finalmente la regla perceptron usando SGD con tasa de aprendizaje $\mu$ es 
$$
\begin{align}
\textbf{w}^{(t+1)} &= \textbf{w}^{(t)} - \mu \frac{d \mathcal{L}(\textbf{w}^{(t)} )}{d \textbf{w}} \nonumber \\
& = \textbf{w}^{(t)} + \begin{cases} \mu d_n \text{concat}(1, u_n) & \\ 0 & \text{en otro caso} \end{cases}
\end{align}
$$
es decir que si el ejemplo está bien clasificado los pesos no se actualizan

### Detalles del algoritmo perceptrón
- En cada iteración se presenta un ejemplo 
- Se dice que se completa una época de entrenamiento cuando se han presentado los $N$ ejemplos
- Presentar los ejemplos en distinto orden en cada época ayuda a evitar sesgos y acelera la convergencia
- Detenemos el entrenamiento cuando todos los ejemplos están bien clasificados o al cumplir un cierto número de épocas sin cambio
- El algoritmo perceptrón está garantizado a converger en tiempo finito si el problema es **linealmente separable**
- Si el problema no es **linealmente separable** la convergencia se puede forzar disminuyendo gradualmente $\mu$


In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
N = 50
u = np.concatenate((np.random.randn(N, 2), 2 + np.random.randn(N, 2)))
d = np.ones(shape=(2*N,)); d[:N] = -1.
w = np.zeros(shape=(3, ))
ax.scatter(u[:N, 0], u[:N, 1]); ax.scatter(u[N:, 0], u[N:, 1])
x_plot = np.linspace(np.amin(u), np.amax(u))
plane = ax.plot(x_plot, -w[0]/(w[2]+1e-10) - x_plot*w[1]/(w[2]+1e-10), 'k-', lw=4, alpha=0.75) 
P = np.random.permutation(2*N)
dot = ax.plot([], [], 'ko', markersize=10)
mu = 1e-5

def update(n):
    global w
    idx = P[n - 2*N*int(n / (2*N))]
    if d[idx]*np.dot(w, np.concatenate((np.array([1]), u[idx]))) <= 0.:
        w = w + mu*d[idx]*np.concatenate((np.array([1]), u[idx]))
    dot[0].set_data(u[idx, 0], u[idx, 1])
    plane[0].set_ydata(-w[0]/(w[2]+1e-10) - x_plot*w[1]/(w[2]+1e-10))
    ax.set_title("Iteration %d" %(n))
        

anim = animation.FuncAnimation(fig, update, frames=300, interval=100, repeat=False, blit=True)


### Más allá del perceptron 

- El modelo de neurona con salida sigmoide se conoce como **regresión logística**
- Tanto el perceptrón como el regresor logístico se pueden extender a más de dos clases: **regresor softmax**
- Conectando varias neuronas en cadena se forma lo que se conoce como una perceptrón multicapa
- El perceptrón multicapa es un ejemplo de **red neuronal artificial**
- Las redes neuronales artificiales se estudiarán en detalle en el curso de **inteligencia artificial**

### Tópicos extra

- [On the Intrinsic Relationship Between the Least Mean Square and Kalman Filters](https://www.commsp.ee.ic.ac.uk/~mandic/LMS_Kalman_IEEE_SPM_2015.pdf)
- [Adaptive notch filter](http://homes.esat.kuleuven.be/~tvanwate/courses/dsp2/1415/DSP2_slides_04_adaptievefiltering.pdf)
- [Reconstruction using Wiener filter](https://wwwmpa.mpa-garching.mpg.de/~ensslin/lectures/Files/Wiener_Filter_Demo_NIFTy3.html)
- [Kalman filter](https://scipy-cookbook.readthedocs.io/items/KalmanFiltering.html)

